In [18]:
import numpy
import analysis
import lsst.afw.table
import lsst.daf.persistence

In [12]:
def loadSExtractor(filename):
    dtype = numpy.dtype([("id", numpy.int64), ("ra", numpy.float64), ("dec", numpy.float64),
                         ("r", numpy.float64), ("i", numpy.float64), ("z", numpy.float64)])
    npCat = numpy.loadtxt(filename, dtype=dtype, usecols=[0, 3, 4, 100, 101, 102])
    npCat["ra"] *= numpy.pi/180
    npCat["dec"] *= numpy.pi/180
    schema = lsst.afw.table.SimpleTable.makeMinimalSchema()
    rKey = schema.addField("r.se", type=float, doc="SExtractor r-band aperture magnitude", units="mag")
    iKey = schema.addField("i.se", type=float, doc="SExtractor i-band aperture magnitude", units="mag")
    zKey = schema.addField("z.se", type=float, doc="SExtractor z-band aperture magnitude", units="mag")
    catalog = lsst.afw.table.SimpleCatalog(schema)
    catalog.reserve(len(npCat))
    for row in npCat:
        catalog.addNew()
    catalog["id"][:] = npCat["id"]
    catalog["coord.ra"][:] = npCat["ra"]
    catalog["coord.dec"][:] = npCat["dec"]
    catalog["r.se"][:] = npCat["r"]
    catalog["i.se"][:] = npCat["i"]
    catalog["z.se"][:] = npCat["z"]
    return catalog

In [13]:
seCat = loadSExtractor("unified_RIZ_catalog_8522.cat")

In [14]:
len(seCat)

180

In [17]:
butler = lsst.daf.persistence.Butler("/home/jbosch/HSC/data/rerun/HSC-1339/baseline")

In [20]:
loader = analysis.CatalogLoader(butler)
hscCat = loader.read(tract=8522, patch="5,5")

In [21]:
help(lsst.afw.table.matchRaDec)

Help on function matchRaDec in module lsst.afw.table.tableLib:

matchRaDec(*args)
    matchRaDec(SimpleCatalog cat1, SimpleCatalog cat2, Angle radius, bool closest=True) -> SimpleMatchVector
    matchRaDec(SimpleCatalog cat1, SimpleCatalog cat2, Angle radius) -> SimpleMatchVector
    matchRaDec(SimpleCatalog cat1, SimpleCatalog cat2, Angle radius, MatchControl mc=lsst::afw::table::MatchControl()) -> SimpleMatchVector
    matchRaDec(SimpleCatalog cat1, SourceCatalog cat2, Angle radius, bool closest=True) -> ReferenceMatchVector
    matchRaDec(SimpleCatalog cat1, SourceCatalog cat2, Angle radius) -> ReferenceMatchVector
    matchRaDec(SimpleCatalog cat1, SourceCatalog cat2, Angle radius, MatchControl mc=lsst::afw::table::MatchControl()) -> ReferenceMatchVector
    matchRaDec(SourceCatalog cat1, SourceCatalog cat2, Angle radius, bool closest=True) -> SourceMatchVector
    matchRaDec(SourceCatalog cat1, SourceCatalog cat2, Angle radius) -> SourceMatchVector
    matchRaDec(SourceCatalog cat

In [28]:
coaddI = butler.get("deepCoadd", tract=8522, patch="5,5", filter="HSC-I", immediate=True)
coaddZ = butler.get("deepCoadd", tract=8522, patch="5,5", filter="HSC-Z", immediate=True)
coaddR = butler.get("deepCoadd", tract=8522, patch="5,5", filter="HSC-R", immediate=True)

In [23]:
import lsst.afw.display.ds9

In [41]:
lsst.afw.display.ds9.mtv(coaddR, frame=0)
lsst.afw.display.ds9.mtv(coaddI, frame=1)
lsst.afw.display.ds9.mtv(coaddZ, frame=2)

In [38]:
def showCatalog(catalog, exposure, symbol=None, color=None, frame=0):
    bbox = lsst.afw.geom.Box2D(exposure.getBBox(lsst.afw.image.PARENT))
    with lsst.afw.display.Buffering():
        for record in catalog:
            point = exposure.getWcs().skyToPixel(record.getCoord())
            if not bbox.contains(point): continue
            point -= lsst.afw.geom.Extent2D(exposure.getXY0())
            if symbol is None:
                lsst.afw.display.ds9.dot(record.getShape(), point.getX(), point.getY(), ctype=color, frame=frame)
            else:
                lsst.afw.display.ds9.dot(symbol, point.getX(), point.getY(), ctype=color, frame=frame)

In [42]:
showCatalog(seCat, coaddI, symbol="+", color="orange", frame=0)
showCatalog(seCat, coaddI, symbol="+", color="orange", frame=1)
showCatalog(seCat, coaddI, symbol="+", color="orange", frame=2)